In [3]:
# ==========================================================
# ESP32 Heater ML Pipeline - Optimized Proactive Regressor (Leakage Fixed)
# ==========================================================
#
# Purpose:
# This script trains a single-stage Random Forest Regressor, now with
# explicit safeguards against Imputation Data Leakage.
#
# Key Fixes:
# 1. Temporal Leakage: Addressed by Run-Aware Train/Test Split.
# 2. Imputation Leakage: Addressed by calculating missing value means ONLY
#    on the training set and applying that mean to the test set.
#
# ==========================================================

import pandas as pd
import numpy as np
import glob
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import joblib

# --- Configuration Settings (ADJUST THESE) ---
CSV_DIRECTORY_PATH = '.'
PREDICTION_WINDOW_STEPS = 10
LAG_STEPS_TEMP = [1, 5, 10]
LAG_STEPS_PWM = [5, 10]
# ---------------------------------------------

MAP_FEATURES = {
    'thrmstr6': 'T_Heater',
    'thrmstrout': 'T_Outside',
    'bme_temp': 'T_Room_BME',
    'pwm': 'PWM_Applied',
    'activity_sim': 'T_Target_Demand'
}

# ==========================================================
# Step 1 & 2: Load Data and Basic Cleaning (Keep NaNs for Imputation)
# ==========================================================
csv_file_pattern = os.path.join(CSV_DIRECTORY_PATH, '*.csv')
all_csv_files = glob.glob(csv_file_pattern)

file_names = [f for f in all_csv_files
              if os.path.basename(f).startswith('Dynamic_') or
                 os.path.basename(f).startswith('Stabilized_')]

if not file_names:
    raise ValueError(f"No CSV files found matching the criteria in: {CSV_DIRECTORY_PATH}")

df_list = []
for f in file_names:
    df = pd.read_csv(f)
    df['source_file'] = os.path.basename(f)
    df_list.append(df)

combined_df = pd.concat(df_list, ignore_index=True)
combined_df.columns = [c.strip().lower() for c in combined_df.columns]
combined_df.rename(columns=MAP_FEATURES, inplace=True)
print(f"Combined DataFrame shape: {combined_df.shape}")

# NOTE: Missing values are NOT filled here. They are filled AFTER the split.

if 'T_Target_Demand' not in combined_df.columns:
    combined_df['T_Target_Demand'] = 25.0

# ==========================================================
# Step 3: Feature Engineering (Lagged & Delta Features)
# ==========================================================
def feature_engineer_time_series(df):
    # This is slightly modified to keep rows with NaNs created by shifting
    # as we will drop them later, but we need the full data for the split.

    df['PWM_Target'] = df['PWM_Applied'].shift(-PREDICTION_WINDOW_STEPS)

    for col in ['T_Heater', 'T_Room_BME']:
        for lag in LAG_STEPS_TEMP:
            df[f'{col}_lag_{lag}min'] = df[col].shift(lag)

    for lag in LAG_STEPS_PWM:
        df[f'PWM_Applied_lag_{lag}min'] = df['PWM_Applied'].shift(lag)

    df['T_Heater_Delta_5min'] = df['T_Heater'] - df['T_Heater'].shift(5)

    # Drop rows where target or lagged features are NaN (created by the shifts)
    df.dropna(subset=['PWM_Target', 'T_Heater_lag_10min', 'PWM_Applied_lag_10min'], inplace=True)
    return df

combined_df_fe = feature_engineer_time_series(combined_df.copy())
print(f"DataFrame shape after feature engineering: {combined_df_fe.shape}")

# ==========================================================
# Step 4: Stage B - Predict Log-Transformed PWM Control (REGRESSION & LEAKAGE FIX)
# ==========================================================

# 1. Define Optimized Feature List
feature_cols_stage_b_optimized = [
    'T_Heater', 'T_Outside', 'T_Room_BME', 'T_Target_Demand', 'T_Heater_Delta_5min',
    'T_Heater_lag_1min', 'T_Heater_lag_5min', 'T_Heater_lag_10min',
    'T_Room_BME_lag_1min', 'T_Room_BME_lag_5min', 'T_Room_BME_lag_10min',
    'PWM_Applied_lag_5min', 'PWM_Applied_lag_10min',
]

X = combined_df_fe[feature_cols_stage_b_optimized]
y = np.log1p(combined_df_fe['PWM_Target'])

# 2. Train/Test Split by Runs
unique_runs = combined_df_fe['source_file'].unique()
train_runs, test_runs = train_test_split(unique_runs, test_size=0.265, random_state=42)

train_mask_b = combined_df_fe['source_file'].isin(train_runs)
test_mask_b = combined_df_fe['source_file'].isin(test_runs)

X_train_b, X_test_b = X[train_mask_b].copy(), X[test_mask_b].copy()
y_train_b, y_test_b = y[train_mask_b].copy(), y[test_mask_b].copy()

# 3. --- CRITICAL LEAKAGE FIX: Impute missing values using only the training set mean ---
print("\nApplying Imputation Leakage Fix...")
imputation_cols = ['T_Heater', 'T_Outside', 'T_Room_BME'] # Features that might have missing values

for col in imputation_cols:
    # 1. Calculate mean ONLY on the training data
    train_mean = X_train_b[col].mean()

    # 2. Impute training set with its own mean
    X_train_b[col] = X_train_b[col].fillna(train_mean)

    # 3. Impute testing set with the training set's mean (NO LEAKAGE)
    X_test_b[col] = X_test_b[col].fillna(train_mean)

print(f"Stage B Training shape: {X_train_b.shape}, Testing shape: {X_test_b.shape}")

# 4. Train Random Forest REGRESSOR
print("\nTraining Stage B Random Forest REGRESSOR...")
rf_stage_b_regressor = RandomForestRegressor(
    n_estimators=100, max_depth=None, random_state=42, n_jobs=-1
)
rf_stage_b_regressor.fit(X_train_b, y_train_b)

# 5. Evaluate Stage B
y_pred_b_log = rf_stage_b_regressor.predict(X_test_b)

# Inverse Transform Predictions for Final MAE
y_pred_b_linear = np.expm1(y_pred_b_log)
y_test_b_linear = np.expm1(y_test_b)

mae_b = mean_absolute_error(y_test_b_linear, y_pred_b_linear)
rmse_b = np.sqrt(mean_squared_error(y_test_b_linear, y_pred_b_linear))
r2_score_linear = r2_score(y_test_b_linear, y_pred_b_linear)

print("\n--- Stage B Test Performance (Log-Transformed Regression) ---")
print(f"Mean Absolute Error (MAE): {mae_b:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse_b:.4f}")
print(f"R-squared (R2, Linear): {r2_score_linear:.4f}")

# 6. Feature Importance and Model Saving
importances = rf_stage_b_regressor.feature_importances_
importance_df = pd.DataFrame({'Feature': feature_cols_stage_b_optimized, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("\nStage B Feature Importances (Top 5):")
print(importance_df.head(5).to_string(index=False))

# Save the final model
model_filename = 'rf_proactive_heater_regressor_final_fixed.joblib'
joblib.dump(rf_stage_b_regressor, model_filename)
print(f"\nFinal Regressor Model saved as: {model_filename}")

print("\n==========================================================")
print("Pipeline Complete: Leakage-Free and Optimized.")
print("==========================================================")








Combined DataFrame shape: (287780, 17)
DataFrame shape after feature engineering: (287760, 28)

Applying Imputation Leakage Fix...
Stage B Training shape: (201638, 13), Testing shape: (86122, 13)

Training Stage B Random Forest REGRESSOR...

--- Stage B Test Performance (Log-Transformed Regression) ---
Mean Absolute Error (MAE): 53.0054
Root Mean Square Error (RMSE): 152.0067
R-squared (R2, Linear): 0.6865

Stage B Feature Importances (Top 5):
             Feature  Importance
PWM_Applied_lag_5min    0.943533
           T_Outside    0.008484
  T_Heater_lag_10min    0.007290
            T_Heater    0.006800
   T_Heater_lag_1min    0.006044

Final Regressor Model saved as: rf_proactive_heater_regressor_final_fixed.joblib

Pipeline Complete: Leakage-Free and Optimized.
